In [12]:
import sqlite3
from autogc_validation.database.schema.create_table import create_table
from autogc_validation.database.schema.schemas import SCHEMAS
from autogc_validation.database.connection.manager import get_connection
import pandas as pd
import logging
import tempfile
import os


In [13]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [14]:
with tempfile.NamedTemporaryFile(suffix=".db", delete=False) as tmpfile:
    db_path = tmpfile.name

logger.info(f"Temporary database path: {db_path}")

def show_tables(conn):
    cursor = conn.execute("SELECT name FROM sqlite_master WHERE type='table';")
    return [row[0] for row in cursor.fetchall()]

# -----------------------------
# Create tables and inspect
# -----------------------------
with get_connection(db_path) as conn:
    # Create tables from SCHEMAS
    for table_name, sql in SCHEMAS.items():
        logger.info(f"Creating table: {table_name}")
        conn.execute(sql.sql)

    tables = show_tables(conn)
    print("Tables in DB:", tables)

# -----------------------------
# Optional: Clean up
# -----------------------------
os.remove(db_path)
logger.info("Temporary database deleted.")

INFO:__main__:Temporary database path: C:\Users\AENGST~1\AppData\Local\Temp\tmpn8snfdq4.db
INFO:__main__:Creating table: schema_version
INFO:__main__:Creating table: voc_info
INFO:__main__:Creating table: sites
INFO:__main__:Creating table: canister_types
INFO:__main__:Creating table: primary_canisters
INFO:__main__:Creating table: primary_canister_concentration
INFO:__main__:Creating table: site_canisters
INFO:__main__:Creating table: mdls
INFO:__main__:Creating table: site_canister_concentration_view
INFO:__main__:Temporary database deleted.


Tables in DB: ['SchemaVersion', 'voc_info', 'sites', 'canister_types', 'primary_canisters', 'primary_canister_concentration', 'site_canisters', 'mdls']


In [10]:
import sqlite3
import pandas as pd
import logging
import tempfile
import os
from autogc_validation.database.management.init_db import initialize_database
from autogc_validation.database.repositories.voc_info import get_all_voc_data
from autogc_validation.database.repositories.sites import insert_site, get_all_sites
from autogc_validation.database.models.site import Site


In [11]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [12]:
with tempfile.NamedTemporaryFile(suffix=".db", delete=False) as tmpfile:
    db_path = tmpfile.name

logger.info(f"Temporary database path: {db_path}")
initialize_database(db_path, force = True)
get_all_voc_data(db_path)



INFO:__main__:Temporary database path: C:\Users\AENGST~1\AppData\Local\Temp\tmpbguxmpxr.db
INFO:autogc_validation.database.management.init_db:Deleted existing database
INFO:autogc_validation.database.management.init_db:Initializing database at C:\Users\AENGST~1\AppData\Local\Temp\tmpbguxmpxr.db
INFO:autogc_validation.database.management.init_db:Creating tables...
INFO:autogc_validation.database.schema.create_table:Created table SchemaVersion
INFO:autogc_validation.database.schema.create_table:Created table voc_info
INFO:autogc_validation.database.schema.create_table:Created table sites
INFO:autogc_validation.database.schema.create_table:Created table canister_types
INFO:autogc_validation.database.schema.create_table:Created table primary_canisters
INFO:autogc_validation.database.schema.create_table:Created table primary_canister_concentration
INFO:autogc_validation.database.schema.create_table:Created table site_canisters
INFO:autogc_validation.database.schema.create_table:Created tabl

[VOCInfo(aqs_code=43202, compound='Ethane', category='Alkane', carbon_count=2, molecular_weight=30.07, column='PLOT', elution_order=1, priority=1),
 VOCInfo(aqs_code=43203, compound='Ethylene', category='Alkene', carbon_count=2, molecular_weight=28.05, column='PLOT', elution_order=2, priority=1),
 VOCInfo(aqs_code=43204, compound='Propane', category='Alkane', carbon_count=3, molecular_weight=44.097, column='PLOT', elution_order=3, priority=1),
 VOCInfo(aqs_code=43205, compound='Propylene', category='Alkene', carbon_count=3, molecular_weight=42.081, column='PLOT', elution_order=4, priority=1),
 VOCInfo(aqs_code=43214, compound='Iso-butane', category='Alkane', carbon_count=4, molecular_weight=58.12, column='PLOT', elution_order=5, priority=1),
 VOCInfo(aqs_code=43212, compound='N-butane', category='Alkane', carbon_count=4, molecular_weight=58.12, column='PLOT', elution_order=6, priority=1),
 VOCInfo(aqs_code=43206, compound='Acetylene', category='Alkyne', carbon_count=2, molecular_weight

In [13]:
s1 = Site(490353014, 'LP', 'Lake Park', 40.709905, -112.008684, '2025-02-06 10:00:00')  
insert_site(db_path, s1)
get_all_sites(db_path)

[Site(site_id=490353014, name_short='LP', name_long='Lake Park', lat=40.709905, long=-112.008684, date_started='2025-02-06 10:00:00')]

In [14]:
os.remove(db_path)
logger.info("Temporary database deleted.")

INFO:__main__:Temporary database deleted.
